# 1. `scipy.fftpack`: rychlá Fourierova transformace

Po práci s maticemi přejdeme ke zpracování signálu. FFT (Fast Fourier Transform) převádí diskrétní signál z časové oblasti do frekvenční oblasti.

V novějším kódu se často používá modul `scipy.fft`; v této lekci zůstáváme kvůli návaznosti u `scipy.fftpack`.

## 1.1 Vytvoření čistého a zašuměného signálu

Připravíme testovací signál složený ze základní frekvence a harmonických složek. Pak k němu přidáme náhodný šum, abychom viděli rozdíl v časové i frekvenční oblasti.

In [ ]:
import numpy as np
from scipy import fftpack
from IPython.display import Audio
import matplotlib.pyplot as plt


In [ ]:
# Nastavení vzorkování
sample_rate = 8000
duration = 1.0

# časová osa (endpoint=False kvůli konzistentní FFT mřížce)
time = np.linspace(0, duration, int(duration * sample_rate), endpoint=False)

# základní frekvence a harmonické
base_frequency = 800
harmonic_2 = 2 * base_frequency
harmonic_3 = 3 * base_frequency

# čistý signál
clean_signal = (
    np.sin(2 * np.pi * base_frequency * time)
    + 0.5 * np.sin(2 * np.pi * harmonic_2 * time)
    + 0.3 * np.sin(2 * np.pi * harmonic_3 * time)
)

# přidání šumu
noisy_signal = clean_signal + 0.5 * np.random.randn(len(clean_signal))

In [ ]:
# čistý signál
Audio(data=clean_signal, rate=sample_rate)

In [ ]:
# zašuměný signál
Audio(data=noisy_signal, rate=sample_rate)

## 1.2 Fourierova transformace a frekvenční spektrum

Nad čistým i zašuměným signálem spočítáme FFT a zobrazíme amplitudové spektrum. V grafu pak uvidíme, které frekvence odpovídají užitečnému signálu a které spíš šumu.

In [ ]:
# aplikujeme FFT
noisy_fft = fftpack.fft(noisy_signal)
clean_fft = fftpack.fft(clean_signal)

# frekvenční osa (vzorkovací perioda je 1 / sample_rate)
frequencies = fftpack.fftfreq(len(clean_signal), d=1 / sample_rate)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))

ax1.plot(time, clean_signal)
ax1.set_title('Časový průběh signálu')
ax1.set_xlabel('Čas [s]')
ax1.set_ylabel('Amplituda')
ax1.set_xlim(0, 0.01)

ax2.plot(frequencies, np.abs(clean_fft))
ax2.set_title('Frekvenční spektrum')
ax2.set_xlabel('Frekvence [Hz]')
ax2.set_ylabel('Amplituda')

plt.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))

ax1.plot(time, noisy_signal)
ax1.set_title('Časový průběh signálu')
ax1.set_xlabel('Čas [s]')
ax1.set_ylabel('Amplituda')
ax1.set_xlim(0, 0.01)

ax2.plot(frequencies, np.abs(noisy_fft))
ax2.set_title('Frekvenční spektrum')
ax2.set_xlabel('Frekvence [Hz]')
ax2.set_ylabel('Amplituda')

plt.tight_layout()
plt.show()

## 1.3 Jednoduché odšumění a návrat do času

V této ukázce použijeme jednoduché prahování amplitudy ve frekvenční oblasti. Jde o demonstrační postup: pro reálná data se běžně používají robustnější filtrační metody.

In [ ]:
# ve spektru oddělíme dominantní složky jednoduchým prahem amplitudy
mask = np.abs(noisy_fft) > 200
filtered_fft = noisy_fft * mask

# návrat do časové oblasti
filtered_signal = np.real(fftpack.ifft(filtered_fft))

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))

ax1.plot(time, filtered_signal)
ax1.set_title('Časový průběh signálu')
ax1.set_xlabel('Čas [s]')
ax1.set_ylabel('Amplituda')
ax1.set_xlim(0, 0.01)

ax2.plot(frequencies, np.abs(filtered_fft))
ax2.set_title('Frekvenční spektrum')
ax2.set_xlabel('Frekvence [Hz]')
ax2.set_ylabel('Amplituda')
ax2.set_xlim(0, 2500)

plt.tight_layout()
plt.show()

In [ ]:
# filtrovaný signál
Audio(data=filtered_signal, rate=sample_rate)